# Overview of saving and restoring models

When you want to save and load variables, the graph, and the graph's metadata--basically, when you want to save or restore your model--we recommend using SavedModel. SavedModel is a language-neutral, recoverable, hermetic serialization format. SavedModel enables higher-level systems and tools to produce, consume, and transform TensorFlow models. TensorFlow provides several mechanisms for interacting with SavedModel, including tf.saved_model APIs, Estimator APIs and a CLI.



## APIs to build and load a SavedModel

The SavedModelBuilder class provides functionality to save multiple MetaGraphDefs. 
* A MetaGraph is a dataflow graph, plus its associated variables, assets, and signatures. 

* A MetaGraphDef is the protocol buffer representation of a MetaGraph. 

* A signature is the set of inputs to and outputs from a graph.



In [7]:
import tensorflow as tf

### Example

[link](https://www.tensorflow.org/serving/serving_basic)

In [22]:
!python mnist_saved_model.py --training_iteration 10 /tmp/saved_model_example

Training model...
Extracting /tmp/train-images-idx3-ubyte.gz
Extracting /tmp/train-labels-idx1-ubyte.gz
Extracting /tmp/t10k-images-idx3-ubyte.gz
Extracting /tmp/t10k-labels-idx1-ubyte.gz
2017-11-20 17:07:04.629290: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
training accuracy 0.6509
Done training!
Exporting trained model to b'/tmp/saved_model_example/1'
Done exporting!


In [24]:
!tree /tmp/saved_model_example/1

/tmp/saved_model_example/1
├── saved_model.pb
└── variables
    ├── variables.data-00000-of-00001
    └── variables.index

1 directory, 3 files


## CLI to inspect and execute SavedModel

Before go on, first introduce a tool for inspecting SavedModel

Overview of commands

The SavedModel CLI supports the following two commands on a MetaGraphDef in a SavedModel:

* show, which shows a computation on a MetaGraphDef in a SavedModel.
* run, which runs a computation on a MetaGraphDef.

In this tutorial, we focus on `show` command.

### Show command

#### Show MetaGraphDef
Because a SavedMode contains one or more `MetaGraphDef`, identified by their tag-sets.
We first check `MetaGraphDef`

In [2]:
!saved_model_cli show --dir /tmp/saved_model_example/1

The given SavedModel contains the following tag-sets:
serve


#### Show SignatureDef
Then we check avaible SignatureDef keys in a particular `MetaGraphDef`

In [3]:
!saved_model_cli show --dir /tmp/saved_model_example/1 --tag_set serve

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "predict_images"
SignatureDef key: "serving_default"


#### Show TensorInfo
To show all inputs and outputs TensorInfo for a specific SignatureDef, pass in the SignatureDef key to signature_def option.

In [4]:
!saved_model_cli show --dir /tmp/saved_model_example/1 --tag_set serve --signature_def predict_images

The given SavedModel SignatureDef contains the following input(s):
inputs['images'] tensor_info:
    dtype: DT_FLOAT
    shape: (-1, 784)
    name: x:0
The given SavedModel SignatureDef contains the following output(s):
outputs['scores'] tensor_info:
    dtype: DT_FLOAT
    shape: (-1, 10)
    name: y:0
Method name is: tensorflow/serving/predict


we can also check all information in a SavedModel

In [5]:
!saved_model_cli show --dir /tmp/saved_model_example/1 --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['predict_images']:
The given SavedModel SignatureDef contains the following input(s):
inputs['images'] tensor_info:
    dtype: DT_FLOAT
    shape: (-1, 784)
    name: x:0
The given SavedModel SignatureDef contains the following output(s):
outputs['scores'] tensor_info:
    dtype: DT_FLOAT
    shape: (-1, 10)
    name: y:0
Method name is: tensorflow/serving/predict

signature_def['serving_default']:
The given SavedModel SignatureDef contains the following input(s):
inputs['inputs'] tensor_info:
    dtype: DT_STRING
    shape: unknown_rank
    name: tf_example:0
The given SavedModel SignatureDef contains the following output(s):
outputs['classes'] tensor_info:
    dtype: DT_STRING
    shape: (-1, 10)
    name: index_to_string_Lookup:0
outputs['scores'] tensor_info:
    dtype: DT_FLOAT
    shape: (-1, 10)
    name: TopKV2:0
Method name is: tensorflow/serving/classify


## Restore

From last section, we know the tensor name in the saved model. Let restore the graph and check the tensor.

In [9]:
export_dir = "/tmp/saved_model_example/1"

graph_restore_saved_model = tf.Graph()
with tf.Session(graph=graph_restore_saved_model) as sess:
    tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], export_dir)
    x = graph_restore_saved_model.get_tensor_by_name("x:0")
    print(x)

INFO:tensorflow:Restoring parameters from b'/tmp/saved_model_example/1/variables/variables'


INFO:tensorflow:Restoring parameters from b'/tmp/saved_model_example/1/variables/variables'


Tensor("x:0", shape=(?, 784), dtype=float32)
